In [1]:
# 0 0 * * * /opt/anaconda3/envs/jupyter_env/bin/ipython /home/slebedev/jhub-analytics/'_ Analytics department'/'[АВТОЗАПУСК]'/'[АВТОЗАПУСК] BA-470 Оплаченные заказы с сайта для метрики.py'

# magic    
get_ipython().run_line_magic('matplotlib', 'inline')
# run files
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/import_libs.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Connecting_.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/my_script.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Date_gen.py')
get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/xx.py')

############################## End ##############################

In [2]:
import requests
import sys
import json
import pandas as pd
import numpy as np
import datetime
import time
import pytz

# Функции для метрики
from Metrika_logs import param_logs
from Metrika_logs import make_logs
from Metrika_logs import param_costs
from Metrika_logs import report_costs

# Функции для БД
from Q478_sql import db_connect
from Q478_sql import query
from Q478_sql import sql_query

1. Выгружаем данные из Я.Метрики за отчетный период

In [3]:
# Определяем период для отчета

report_date = datetime.datetime.now() - datetime.timedelta(days=9)

date_from = (report_date.date() - datetime.timedelta(days=29)).strftime("%Y-%m-%d")
date_till = report_date.date().strftime("%Y-%m-%d") # Дата окончания отчета

# Рекламные аккаунты Я.Директа

client_logins = ['e2e4online18', 'e2e4vendor', 'e2e4gender']

# Определяем цели метрики

targets = {'Оформление заказа на сайте': 45333839, 'CRM: Заказ создан': 296990514}#, 'CRM: Заказ оплачен': 296990523}


In [4]:
# Получаем параметры запроса для выгрузки логов

metrika_headers, metrika_url, payload, count, field_rus = param_logs(date_from, date_till)

# Выгружаем логи с сервера Яндекса в датасет

df_logs = make_logs(metrika_headers, metrika_url, payload, count, field_rus)

1. Проверяем возможность создания логов на сервере Яндекса
2. Отправляем запрос на создание логов на сервер Яндекса
3. Проверяем статус запроса 31765491 на сервере Яндекса
4. Логи готовы (2 партии), скачиваем их с сервера Яндекса
5. Логи скачены, удаляем их с сервера Яндекса
6. Логи удалены с сервера Яндекса


In [44]:
df_log_test = df_logs.copy()

In [59]:
sourse =  list(df_logs_ord['Источник трафика'].unique())
sourse

['direct',
 'organic',
 'internal',
 'ad',
 'referral',
 'social',
 'messenger',
 'email']

In [60]:
system = list(df_logs_ord['Рекламная система'].unique())
system

['ya_undefined',
 'ya_direct',
 'unknown',
 'admitad',
 'target_mail_ru',
 'google_adwords',
 'youtube',
 'price_ru']

In [51]:
lst = []
for i in sourse:
    ls = []
    df = df_log_test[df_log_test['Источник трафика'] == i]
    n = len(df)
    lst.append(n)
    for j in system:
        d = df[df['Рекламная система'] == j]
        k = len(d)
        ls.append(k)
    lst.append(ls)
    

In [57]:
df

,ID визита,Дата и время визита,Город,ID пользователя,ID целей,Время достижения цели,ID заказов,Источник трафика,Рекламная система,Кампания ЯД,Название кампании ЯД,Openstat Ad,Метка from,UTM Campaign,UTM Content,UTM Medium,UTM Source,UTM Term
8,4343032507928674549,2023-07-30 21:02:34,Novosibirsk,1689624679954116037,[],[],[],internal,ya_undefined,,0,,,,,,,
9,4448100963581427887,2023-08-04 12:22:38,Novosibirsk,1614565293674678304,[],[],[],internal,ya_undefined,77386528,Товарная кампания / Новосибирск,,,,,,,
32,4672032661702180867,2023-08-14 09:39:50,Novosibirsk,1681449387646892072,[41613373],[\'2023-08-14 09:39:50\'],[''],internal,ya_undefined,77386528,Товарная кампания / Новосибирск,,,,,,,
37,4512677874302451963,2023-08-07 08:48:19,Novokuznetsk,1686200209198939609,[],[],[],internal,ya_undefined,,0,,,,,,,
55,4014030171224408101,2023-07-16 08:25:09,Novokuznetsk,1612768153965884766,"[31102120,31102120]","[\'2023-07-16 08:25:18\',\'2023-07-16 08:26:25\']","['','']",internal,ya_undefined,80385223,Товарная кампания / Новокузнецк,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407558,4599035553810219259,2023-08-11 04:18:48,Kemerovo,15647334061071255577,[],[],[],internal,ya_undefined,,0,,,,,,,
407560,4600654952411431101,2023-08-11 06:01:45,Omsk,1691574252117454497,[],[],[],internal,ya_undefined,,0,,,,,,,
407561,4076438608217899071,2023-07-19 02:32:59,Ryazan,160618796978391249,[],[],[],internal,ya_undefined,,0,,,,,,,
407562,4419937955870671234,2023-08-03 06:32:05,Novosibirsk,1577680761150667560,[],[],[],internal,ya_undefined,77386528,Товарная кампания / Новосибирск,,,,,,,


In [58]:
df = df_log_test[df_log_test['Источник трафика'] == sourse[1]]
d = df[df['Рекламная система'] == system[3]]
d 

,ID визита,Дата и время визита,Город,ID пользователя,ID целей,Время достижения цели,ID заказов,Источник трафика,Рекламная система,Кампания ЯД,Название кампании ЯД,Openstat Ad,Метка from,UTM Campaign,UTM Content,UTM Medium,UTM Source,UTM Term
116,4169188695148855526,2023-07-23 04:49:52,Stavropol,1690076992788970161,[],[],[],ad,admitad,,0,,,1872658,,cpa,admitad,
266,4026703420644655389,2023-07-16 21:50:54,Nalchik,1689533455448365815,[],[],[],ad,admitad,,0,,,1872658,,cpa,admitad,
372,4027609676217319603,2023-07-16 22:48:31,,168818726098279343,[],[],[],ad,admitad,,0,,,539733,,cpa,admitad,
466,4474044457992847437,2023-08-05 15:52:05,Rostov-na-Donu,1691239956693814927,[],[],[],ad,admitad,,0,,,539733,,cpa,admitad,
625,4114955720043266180,2023-07-20 19:21:50,Moscow,1689870110840154352,[],[],[],ad,admitad,,0,,,232179,,cpa,admitad,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407146,4470105221199823024,2023-08-05 11:41:38,,1661613769291682241,[],[],[],ad,admitad,,0,,,539733,,cpa,admitad,
407214,4319697434840400081,2023-07-29 20:18:57,Krasnodar,1673609474313261782,[],[],[],ad,admitad,,0,,,539733,,cpa,admitad,
407257,4022034821274140980,2023-07-16 16:54:05,,1689515644551036195,[178565533],[\'2023-07-16 16:54:11\'],[''],ad,admitad,,0,,,313278,,cpa,admitad,
407350,4613260258534490245,2023-08-11 19:23:11,Moscow,169177099159813224,[],[],[],ad,admitad,,0,,,539733,,cpa,admitad,


In [52]:
lst

[95733,
 [95733, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 125087,
 [11684, 96570, 11274, 4249, 235, 726, 136, 194, 18, 1],
 128097,
 [128097, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 48754,
 [48754, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 8111,
 [8111, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1017,
 [1017, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 244,
 [244, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 489,
 [489, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 16,
 [16, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 5,
 [5, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 30,
 [30, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [23]:
df_logs_ord

,ID визита,Дата и время визита,Город,ID пользователя,ID целей,Время достижения цели,ID заказов,Источник трафика,Рекламная система,Кампания ЯД,Название кампании ЯД,Openstat Ad,Метка from,UTM Campaign,UTM Content,UTM Medium,UTM Source,UTM Term
18,4316911052586221790,2023-07-29 21:21:48,Domodedovo,168852075715334530,"[178565542, 178565533, 178565533, 178565533, 1...","[2023-07-29 17:23:18, 2023-07-29 17:23:19, 202...",[7452959],direct,ya_undefined,,0,,,,,,,
28,4357034356632977544,2023-07-31 15:52:47,Tyumen,1678787549462987349,"[178565533, 178565542, 178565542, 178565533, 1...","[2023-07-31 11:53:50, 2023-07-31 11:53:50, 202...",[7455975],organic,ya_undefined,87570445,eLama | Сайт | Тюмень | Товарная,,,,,,,
33,4085265897159655844,2023-07-19 15:54:12,Novosibirsk,1684470878736258783,"[31102120, 41070295, 31102120, 41070295, 31102...","[2023-07-19 11:56:16, 2023-07-19 11:56:16, 202...",[7433642],organic,ya_undefined,37359796,Поиск / Динамика / Новосибирск,,,,,,,
87,4190647630010777630,2023-07-24 07:34:12,Khabarovsk,1673226444155335331,"[259371044, 259370017, 38968693, 45333839, 253...","[2023-07-24 03:34:17, 2023-07-24 03:34:35, 202...",[7440346],organic,ya_undefined,,0,,,,,,,
127,4040780890915995678,2023-07-17 16:45:55,Tomsk,1650882531434132474,"[38968693, 45333839, 253251659, 178565542, 296...","[2023-07-17 12:49:32, 2023-07-17 12:49:41, 202...",[7428520],direct,ya_undefined,77431097,Товарная кампания / Томск,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407476,4669859999048007732,2023-08-14 11:21:42,Novosibirsk,1640245805927415937,"[38968693, 38968693, 45333839, 253250927]","[2023-08-14 07:29:40, 2023-08-14 07:29:52, 202...",[7471931],direct,ya_undefined,37194028,Смарт-баннеры / Новосибирск,,,,,,,
407479,4361871107446014064,2023-07-31 21:00:17,Moscow,1679928222205227993,"[178565533, 178565542, 38968693, 178565542, 45...","[2023-07-31 17:00:19, 2023-07-31 17:00:19, 202...",[7456626],referral,ya_undefined,80178380,Товарная кампания / Москва new,,,,,,,
407504,4442752025328877849,2023-08-04 10:42:34,,1678849243434336385,"[45333839, 253251659, 178565542, 178565542, 17...","[2023-08-04 06:46:31, 2023-08-04 06:46:31, 202...",[7465289],internal,ya_undefined,,0,,,,,,,
407530,4221119622921584665,2023-07-25 15:51:33,Novokuznetsk,1667973619165300063,"[31102120, 38968693, 45333839, 253250927]","[2023-07-25 11:52:03, 2023-07-25 11:52:06, 202...",[7441248],direct,ya_undefined,83277664,Товарная кампания / Категории / Новокузнецк,,,,,,,


In [ ]:
# Преобразование типа данных полей таблицы Я.Метрики 

df_logs['Время достижения цели'] = df_logs['Время достижения цели'].str.replace("\\", "")

for i in ['ID целей',
        'Время достижения цели',
        'ID заказов']:
    df_logs[i] = df_logs[i].apply(lambda x: eval(x))

# Фильтруем уникальные позиции заказов 

def unique_ord(row):
    un = set(row)
    l = list(un)
    try:
        l.remove('')
    except:
        l
    return l

df_logs['ID заказов'] = df_logs['ID заказов'].apply(unique_ord)

# Оставляем только целевые визиты

for key, value in targets.items():
    df_logs[key] = df_logs['ID целей'].apply(lambda x: 'order' if value in x else np.nan)
    
df_logs = df_logs[df_logs['Оформление заказа на сайте'].notna() | df_logs['CRM: Заказ создан'].notna()]# | df_logs['CRM: Заказ оплачен'].notna()]
df_logs.drop(columns=['Оформление заказа на сайте', 'CRM: Заказ создан'], inplace=True)

# Выделяем цели ЯМ, касающиеся только оформления заказа на сайте

df_logs = df_logs.explode(['ID целей', 'Время достижения цели'])
df_logs = df_logs[df_logs['ID целей'].isin(list(targets.values()))]

# Приводим даты и время к часовому поясу НСК+7

tz_3 = pytz.timezone("Europe/Moscow")
tz_7 = pytz.timezone("Asia/Novosibirsk")

def interval(row):
    lst = []
    for i in range(-3, 6):
        x = datetime.datetime.strftime(tz_3.localize(datetime.datetime.strptime(row, '%Y-%m-%d %H:%M:%S')).astimezone(tz_7) - datetime.timedelta(seconds=i), '%Y-%m-%d %H:%M:%S')
        lst.append(x)
    return lst

df_logs['interval'] = df_logs['Время достижения цели'].apply(interval)

for i in ['Дата и время визита', 'Время достижения цели']:
    df_logs[i] = df_logs[i].apply(lambda x: datetime.datetime.strftime(tz_3.localize(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')).astimezone(tz_7), '%Y-%m-%d %H:%M:%S'))
    


In [ ]:
# Выгружаем затраты с сервера Яндекса в датасет

download_status, df_costs = report_costs(date_from, date_till, client_logins)
download_status

In [ ]:
# Преобразование таблицы расходов Я.Метрики

df_costs.columns = ['Источник трафика', 'Кампания ЯД', 'Название кампании', 'Расходы по рекламной кампании']
df_costs = df_costs.drop_duplicates()

2. Выгружаем заказы из БД за отчетный период

In [ ]:
# Формируем SQL запрос

sql_query = sql_query(date_from, date_till)

In [ ]:
# Получаем данные из БД

df_sql = query(sql_query)

In [ ]:
df_sql_test = df_sql.copy()

In [ ]:
df_sql_test

In [ ]:
# Преобразование типа данных таблицы заказов из БД

df_sql['order_date'] = df_sql['order_date'].apply(lambda x: datetime.datetime.strftime(x.astimezone(tz_7), '%Y-%m-%d %H:%M:%S'))
df_sql['y_client_ids'] = df_sql['y_client_ids'].apply(lambda x: None if x == 'null' else x)
for i in range(9, 14):
    df_sql.iloc[:, i] = df_sql.iloc[:, i].fillna(0)

3. Объединяем заказы из БД и заказы из Я.Метрики

In [ ]:
# Объединяем клиентов из БД и метрики
df_merge = df_sql.merge(df_logs, how='left', left_on='y_client_ids', right_on='ID пользователя')
df_merge = df_merge[df_merge['ID пользователя'].notna()]

# Сводим заказы по времени оформления
df_merge['in'] = df_merge.apply(lambda x: x['order_date'] in x['interval'], axis=1)
df_merge = df_merge[df_merge['in']]
df_merge = df_merge[~df_merge.duplicated(subset=["id"])]

# Объединяем заказы из БД и сведенные заказы в одну таблицу
df_merge2 = df_sql.merge(df_merge, how='left', on='id')
df_merge2 = df_merge2.merge(df_costs[df_costs['Кампания ЯД'].notna()], how='left', on='Кампания ЯД')
df_merge2['Кампания ЯД'] = df_merge2['Кампания ЯД'].apply(lambda x: int(x) if x != '' and pd.notna(x) else np.nan).astype('Int64')
df_merge2['Расходы по рекламной кампании'] = df_merge2['Расходы по рекламной кампании'].fillna(0)

In [ ]:
# Расходы по рекламным системам

ya_direct = sum(df_costs[df_costs['Источник трафика'] == 'Yandex.Direct']['Расходы по рекламной кампании'])
vk = sum(df_costs[df_costs['Источник трафика'] == 'vk']['Расходы по рекламной кампании']) + sum(df_costs[df_costs['Источник трафика'] == 'elama.vk_ads']['Расходы по рекламной кампании'])
retailrocket = sum(df_costs[df_costs['Источник трафика'] == 'retailrocket']['Расходы по рекламной кампании'])
news = sum(df_costs[df_costs['Источник трафика'] == '3dnews']['Расходы по рекламной кампании'])
gis = sum(df_costs[df_costs['Источник трафика'] == '2gis']['Расходы по рекламной кампании'])
admitad = sum(df_costs[df_costs['Источник трафика'] == 'admitad']['Расходы по рекламной кампании'])
regmarkets = sum(df_costs[df_costs['Источник трафика'] == 'regmarkets.ru']['Расходы по рекламной кампании'])
price_ru = sum(df_costs[df_costs['Источник трафика'] == 'priceru']['Расходы по рекламной кампании'])

def spends(row):
    if row['Рекламная система'] == 'ya_direct' or row['Рекламная система'] == 'ya_undefined' or pd.notna(row['Кампания ЯД']):
        x = round(ya_direct, 1)    
    elif row['Рекламная система'] == 'vkontakte':
        x = round(vk, 1)
    elif row['Рекламная система'] == 'admitad':
        x = round(admitad, 1)
    elif row['Рекламная система'] == 'market':
        x = round(regmarkets, 1)
    elif row['Рекламная система'] == 'price_ru':
        x = round(price_ru, 1)
    else:
        x = 0
    return x

df_merge2['Расходы по рекламному каналу'] = df_merge2.apply(spends, axis = 1)

In [ ]:
# Рекламные затраты на заказ

# количество заказов 
ya_orders = len(df_merge2[((df_merge2['Рекламная система'] == 'ya_direct') | (df_merge2['Рекламная система'] == 'ya_undefined') | (pd.notna(df_merge2['Кампания ЯД'])))])    
vk_orders = len(df_merge2[(df_merge2['Рекламная система'] == 'vkontakte') & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
adm_orders = len(df_merge2[(df_merge2['Рекламная система'] == 'admitad') & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
mar_orders = len(df_merge2[(df_merge2['Рекламная система'] == 'market') & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
pr_orders = len(df_merge2[(df_merge2['Рекламная система'] == 'price_ru') & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])

def spend_order(row):
    if row['Рекламная система'] == 'ya_direct' or row['Рекламная система'] == 'ya_undefined' or pd.notna(row['Кампания ЯД']):
        x = round(row['Расходы по рекламному каналу']/ya_orders, 1)    
    elif row['Рекламная система'] == 'vkontakte' and pd.isna(row['Кампания ЯД']):
        x = round(row['Расходы по рекламному каналу']/vk_orders, 1)  
    elif row['Рекламная система'] == 'admitad' and pd.isna(row['Кампания ЯД']):
        x = round(row['Расходы по рекламному каналу']/adm_orders, 1)  
    elif row['Рекламная система'] == 'market' and pd.isna(row['Кампания ЯД']):
        x = round(row['Расходы по рекламному каналу']/mar_orders, 1)  
    elif row['Рекламная система'] == 'price_ru' and pd.isna(row['Кампания ЯД']):
        x = round(row['Расходы по рекламному каналу']/pr_orders, 1)
    else:
        x = 0
    return x

df_merge2['Рекламные затраты на заказ'] = df_merge2.apply(spend_order, axis = 1)


In [ ]:
# Рекламные затраты на выкупленный заказ

# количество выкупов
ya_ship_ord = len(df_merge2[((df_merge2['Рекламная система'] == 'ya_direct') | (df_merge2['Рекламная система'] == 'ya_undefined') | (pd.notna(df_merge2['Кампания ЯД']))) & (df_merge2['sum_payed_x'] != 0) & (pd.notna(df_merge2['Рекламная система']))])    
vk_ship_ord = len(df_merge2[(df_merge2['Рекламная система'] == 'vkontakte') & (df_merge2['sum_payed_x'] != 0) & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
adm_ship_ord = len(df_merge2[(df_merge2['Рекламная система'] == 'admitad') & (df_merge2['sum_payed_x'] != 0) & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
mar_ship_ord = len(df_merge2[(df_merge2['Рекламная система'] == 'market') & (df_merge2['sum_payed_x'] != 0) & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])
pr_ship_ord = len(df_merge2[(df_merge2['Рекламная система'] == 'price_ru') & (df_merge2['sum_payed_x'] != 0) & (pd.notna(df_merge2['Рекламная система'])) & (pd.isna(df_merge2['Кампания ЯД']))])

def ship_order(row):
    if (row['Рекламная система'] == 'ya_direct' or row['Рекламная система'] == 'ya_undefined' or pd.notna(row['Кампания ЯД'])) and row['sum_payed_x'] != 0:
        x = round(row['Расходы по рекламному каналу']/ya_ship_ord, 1)    
    elif row['Рекламная система'] == 'vkontakte' and pd.isna(row['Кампания ЯД']) and row['sum_payed_x'] != 0:
        x = round(row['Расходы по рекламному каналу']/vk_ship_ord, 1)  
    elif row['Рекламная система'] == 'admitad' and pd.isna(row['Кампания ЯД']) and row['sum_payed_x'] != 0:
        x = round(row['Расходы по рекламному каналу']/adm_ship_ord, 1)  
    elif row['Рекламная система'] == 'market' and pd.isna(row['Кампания ЯД']) and row['sum_payed_x'] != 0:
        x = round(row['Расходы по рекламному каналу']/mar_ship_ord, 1)  
    elif row['Рекламная система'] == 'price_ru' and pd.isna(row['Кампания ЯД']) and row['sum_payed_x'] != 0:
        x = round(row['Расходы по рекламному каналу']/pr_ship_ord, 1)
    else:
        x = 0
    return x

df_merge2['Рекламные затраты на выкупленный заказ'] = df_merge2.apply(ship_order, axis = 1)

In [ ]:
# Рассчет маржинальности

df_merge2['Маржа чистая - рекламные затраты на заказ'] = df_merge2['revenue_x'] - df_merge2['Рекламные затраты на заказ']
df_merge2['Маржа чистая - рекламные затраты на выкупленные заказы'] = (df_merge2['sum_payed_x'] - df_merge2['Рекламные затраты на выкупленный заказ']).fillna(0)

In [ ]:

# подготавливаем таблицу к загрузке в облако
df = df_merge2[['id', 'order_status_x', 'db_client_ids_x', 'y_client_ids_x', 
                'order_date_x', 'revenue_x', 'item_count_x', 'marginality_x', 'sum_shipped_x', 
                'ship_item_count_x', 'sum_payed_x', 'margin_x', 'Город', 'client_sity_x', 
                'Рекламная система', 'Кампания ЯД', 'Расходы по рекламному каналу', 
                'Расходы по рекламной кампании', 'Рекламные затраты на заказ', 
                'Маржа чистая - рекламные затраты на заказ', 'Рекламные затраты на выкупленный заказ', 
                'Маржа чистая - рекламные затраты на выкупленные заказы']]

df.columns = ['ID заказа ', 'Статус заказа', 'ID клиента е2е4', 'ID клиента Яндекс', 
              'Дата оформления заказа', 'Сумма оформленного заказа', 'Кол-во позиций в заказе', 
              'Маржинальность заказа ', 'Сумма отгруженного заказа', 'Кол-во позиций отгруженного заказа ',
              'Сумма оплаты', 'Маржа чистая', 'Рекламный город', 'Город клиента', 
              'Рекламная система', 'Кампания ЯД', 'Расходы по рекламному каналу', 
              'Расходы по рекламной кампании', 'Рекламные затраты на заказ', 
              'Маржа чистая - рекламные затраты на заказ', 'Рекламные затраты на выкупленный заказ', 
              'Маржа чистая - рекламные затраты на выкупленные заказы']


4. Загружаем отчет в облачное хранилище 

In [19]:
# Создаем адрес загрузки

Amazon_owncloud = owncloud.Client('https://nextcloud.e2e4.ru')
Amazon_owncloud.login(loginAtlassianjhubAdmin, passwordAtlassianjhubAdmin)

AmazonS3_folder_Public = r'/Public/Отчеты (выгрузка jupyter notebooks)'
AmazonS3_path_to_file = r'/Рекламные затраты на заказы из метрики/Тестовая выгрузка 16.07 - 14.08/'

AmazonS3_full_path_customer = AmazonS3_folder_Public + AmazonS3_path_to_file + r'Логи метрики целевые'


In [20]:
df_logs_ord.to_excel(temp_tmp + AmazonS3_full_path_customer.split('/')[-1] + '.xlsx',  index=False, encoding='cp-1251') #   to_excel

try:
    Amazon_owncloud.put_file( AmazonS3_full_path_customer + '.xlsx', temp_tmp + AmazonS3_full_path_customer.split('/')[-1] + '.xlsx' )
    print( f"✔ Файл записан в облако\n{AmazonS3_full_path_customer.split('/')[-1]}.xlsx" )  
    os.remove(temp_tmp + AmazonS3_full_path_customer.split('/')[-1] + '.xlsx' )
    print( '✔ os.remove' )
except:
    print( 'Возникли проблемы при записи в облако!' )

✔ Файл записан в облако
Логи метрики целевые.xlsx
✔ os.remove
